---
# Projectr : 하루시작 지하철 혼잡도 분석
### Description : 
- <a><span style = "color : #FFBE98">**서울특별시 지하철 혼잡도 관련 News Crwaling**</a>
    
### Author : Zenzen
### Date : 2024. 06. 04. (Tue) ~
### Detail : 
### Update: 
- 2024.06.04 zen : NAVER에서 News Data 가져오기 <br><br>

# NAVER News Crawler (날짜, Link, 제목, 본문)

## Import Library Package for Crwaling

In [6]:
from bs4 import BeautifulSoup
import requests
import re
import datetime
from tqdm import tqdm # tqdm Moduel은 진행 상황을 체크하기 위한 Moudel임.
import sys

# Server에 부하를 주지 않기 위한 Crawling 속도 제한용 Library Package
import random
import time

> 랜덤 시간을 사용하는 이유는 일정한 시간 간격으로 요청을 보내면 규칙성이 있어 탐지될 수 있기 때문입니다. <br><br>
>> 랜덤 시간은 <span style = "color : #F7CAC9">**0.5초에서 2초 사이의 값**</span>을 사용하는 것이 일반적입니다.

## Create a function for Crwaling

In [7]:
# Page url 형식에 맞게 바꾸어 주는 함수 만들기
  ## 입력된 수를 1, 11, 21, 31 ...만들어 주는 함수
def makePgNum (num) :
  if num == 1 :
    return num
  elif num == 0 :
    return num + 1
  else :
    return num + 9 * (num - 1)

# Crawling할 url 생성하는 함수 만들기 (검색어, Crawling 시작 Page, Crawling 종료 Page)
def makeUrl (search, start_pg, end_pg) :
    if start_pg == end_pg :
      start_page = makePgNum(start_pg)
      url = "https://search.naver.com/search.naver?where=news&sm=tab_pge&query=" + search + "&start=" + str(start_page)
      print("생성url: ", url)

      return url

    else :
      urls = []
      for i in range(start_pg, end_pg + 1) :
        page = makePgNum(i)
        url = "https://search.naver.com/search.naver?where=news&sm=tab_pge&query=" + search + "&start=" + str(page)
        urls.append(url)
      print("생성url: ", urls)

      return urls    

# html에서 원하는 속성 추출하는 함수 만들기 (기사, 추출하려는 속성값)
def news_attrs_crawler (articles, attrs) :
  attrs_content = []
  for i in articles :
    attrs_content.append(i.attrs[attrs])
    #***********************************#
    time.sleep(random.uniform(0.5, 2.0))
    #***********************************#
    
  return attrs_content

# ConnectionError방지
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) Chrome/98.0.4758.102"}

# html 생성해서 기사 Crawling하는 함수 만들기(url): Link를 반환
def articles_crawler (url) :
  # html 불러오기
  original_html = requests.get(i, headers = headers)
  #***********************************#
  time.sleep(random.uniform(0.5, 2.0))
  #***********************************#
  html = BeautifulSoup(original_html.text, "html.parser")

  url_naver = html.select("div.group_news > ul.list_news > li div.news_area > div.news_info > div.info_group > a.info")
  url = news_attrs_crawler(url_naver, 'href')
  
  return url

## Crawling할 NAVER News URL 추출하기

In [8]:
##### News Crawling 시작 #####

# 검색어 입력
search = input("검색할 Keyword를 입력해주세요:")
# 검색 시작할 Page 입력
page = int(input("\nCrawling할 시작 Page를 입력해주세요. ex)1(숫자만입력):")) # ex)1 =1페이지,2=2페이지...
print("\nCrawling할 시작 Page: ", page, "Page")
# 검색 종료할 Page 입력
page2 = int(input("\nCrawling할 종료 Page를 입력해주세요. ex)1(숫자만입력):")) # ex)1 =1페이지,2=2페이지...
print("\nCrawling할 종료 Page: ", page2, "Page")


# NAVER url 생성
url = makeUrl(search, page, page2)

# News Crawler 실행
news_titles = []
news_url = []
news_contents = []
news_dates = []

for i in url :
    url = articles_crawler(url)
    news_url.append(url)


# 제목, Link, 내용 1차원 List로 꺼내는 함수 생성
def makeList (newlist, content) :
    for i in content :
        for j in i :
            newlist.append(j)

    return newlist

    
# 제목, Link, 내용 담을 List 생성
news_url_1 = []

# 1차원 List로 만들기 (내용 제외)
makeList(news_url_1, news_url)

# NAVER 뉴스만 남기기
final_urls = []

for i in tqdm(range(len(news_url_1))) :
    if "news.naver.com" in news_url_1[i] :
        final_urls.append(news_url_1[i])
    else :
        pass


Crawling할 시작 Page:  1 Page

Crawling할 종료 Page:  2 Page
생성url:  ['https://search.naver.com/search.naver?where=news&sm=tab_pge&query=서울지하철혼잡AI예측&start=1', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=서울지하철혼잡AI예측&start=11']


100%|██████████| 28/28 [00:00<00:00, 362470.72it/s]


## News 본문 및 날짜 Crawling하기

In [9]:
##### News 내용 Crawling #####
for i in tqdm(final_urls) :
    #각 기사 html get하기
    news = requests.get(i, headers=headers)
    #***********************************#
    time.sleep(random.uniform(0.5, 2.0))
    #***********************************#
    news_html = BeautifulSoup(news.text, "html.parser")


    # News 제목 가져오기
    title = news_html.select_one("#ct > div.media_end_head.go_trans > div.media_end_head_title > h2")
    if title == None :
        title = news_html.select_one("#content > div.end_ct > div > h2")
    
    # News 본문 가져오기
    content = news_html.select("article#dic_area")
    if content == [] :
        content = news_html.select("#articeBody")
        
    # 기사 Text만 가져오기
    # List 합치기
    content = ''.join(str(content))

    # html Tag 제거 및 Text 다듬기
    pattern1 = '<[^>]*>'
    title = re.sub(pattern = pattern1, repl = '', string = str(title))
    content = re.sub(pattern = pattern1, repl = '', string = content)
    pattern2 = """[\n\n\n\n\n// flash 오류를 우회하기 위한 함수 추가\nfunction _flash_removeCallback() {}"""
    content = content.replace(pattern2, '')

    news_titles.append(title)
    news_contents.append(content)

    try :
        html_date = news_html.select_one("div#ct> div.media_end_head.go_trans > div.media_end_head_info.nv_notrans > div.media_end_head_info_datestamp > div > span")
        news_date = html_date.attrs['data-date-time']
        
    except AttributeError :
        news_date = news_html.select_one("#content > div.end_ct > div > div.article_info > span > em")
        news_date = re.sub(pattern = pattern1, repl = '', string = str(news_date))

    # 날짜 가져오기
    news_dates.append(news_date)

print("검색된 기사 갯수: 총 ", (page2 + 1 - page) * 10, '개')
print("\n[뉴스 제목]")
print(news_titles)
print("\n[뉴스 링크]")
print(final_urls)
print("\n[뉴스 내용]")
print(news_contents)

print()

print('news_title: ', len(news_titles))
print('news_url: ', len(final_urls))
print('news_contents: ', len(news_contents))
print('news_dates: ', len(news_dates))

100%|██████████| 8/8 [00:12<00:00,  1.54s/it]

검색된 기사 갯수: 총  20 개

[뉴스 제목]
['지하철역 승강장 혼잡도 실시간 예측한다', '‘혼잡률 200% 육박’ 서울지하철…AI가 ‘심각’ 판단하면 진입 통제', "의자 없애고 혼잡도 실시간 분석…서울 '지옥철' 오명 벗나", '김혜지 서울시의원, 지하철역 혼잡도…다양한 개선대책 추진', 'SKT "서울 지하철 1~8호선 혼잡도 미리 확인한다"', '[기고]빅데이터 기반 지하철 쾌적 경로 서비스 제공해야', '국민 안전 지평 넓힌 공공데이터 활용[포럼]', '핼러윈 주말, 홍대에 최대 7만명 모인다…마포구, 인파관리에 2850명 투입']

[뉴스 링크]
['https://n.news.naver.com/mnews/article/018/0005610128?sid=100', 'https://n.news.naver.com/mnews/article/056/0011594450?sid=102', 'https://n.news.naver.com/mnews/article/421/0007151523?sid=102', 'https://n.news.naver.com/mnews/article/081/0003371347?sid=102', 'https://n.news.naver.com/mnews/article/629/0000042117?sid=105', 'https://n.news.naver.com/mnews/article/030/0002739884?sid=105', 'https://n.news.naver.com/mnews/article/021/0002609200?sid=110', 'https://n.news.naver.com/mnews/article/081/0003403809?sid=102']

[뉴스 내용]
["[\n행안부, 'AI 기반 지하철 승강장 혼잡도 예측 모델' 개발서울지하철 2개 역 시범 적용...정확도 90%, 표준 모델화 추진[이데일리 이연호 기자] 정부가 지하철역 승강장의 혼잡 상황을 실시간으로 파악할 수 있는 인공지능(AI) 기반 데이터 분석 모델을 개발해

## DataFrame 만들고 CSV File로 저장하기

In [10]:
##### DataFrame으로 만들기 #####
import pandas as pd

# DataFrame 만들기
news_df = pd.DataFrame(
  {
    'date' : news_dates,
    'title' : news_titles,
    'link' : final_urls,
    'content' : news_contents,
  }
)

news_df

# 중복 Row(행) 지우기
news_df = news_df.drop_duplicates(keep = 'first', ignore_index = True)
print("중복 제거 후 행 개수: ", len(news_df))

# Save to DataFrame
now = datetime.datetime.now() 
news_df.to_csv("../Data/Crawling_data/{}_{}.csv".format(search, now.strftime('%Y%m%d_%H시%M분%S초')), encoding = 'utf-8-sig', index=False)

중복 제거 후 행 개수:  8
